In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from datetime import datetime as dt
import os

def getNow():
    now       = dt.now().strftime('%Y-%m-%d--%H-%M-%S')
    nowFolder = os.path.join('./logFolder', now)
    return nowFolder

/Users/user/Library/Python/3.6/lib/python/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
N = 1000
X = np.random.rand(N, 2)
y = 2*X[:, 0] + 3*X[:, 1] + 4 + 0.1*(np.random.rand(N)-0.5)
y = y.reshape((-1, 1))
tMask = np.random.choice([True, False], (N,), p=[0.8, 0.2])

In [3]:
with tf.variable_scope('trainData'):
    inpTrain = tf.placeholder(dtype=tf.float32, shape=(None, 2), name='inpTrain')
    outTrain = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='outTrain')

with tf.variable_scope('testData'):
    inpTest = tf.placeholder(dtype=tf.float32, shape=(None, 2), name='inpTest')
    outTest = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='outTest')

with tf.variable_scope('model'):
    W = tf.Variable(initial_value=np.random.rand(2, 1), name='W', dtype=tf.float32)
    b = tf.Variable(initial_value=0, name='b', dtype=tf.float32)
    
    # Add summaries 
    # ---------------------
    tf.summary.scalar('bSummary', b)
    tf.summary.histogram('WSummary', W)
    tf.summary.scalar('WMin', tf.reduce_min(W))
    tf.summary.scalar('WMax', tf.reduce_max(W))

    with tf.variable_scope('train'):
        yHatTrain = tf.matmul(inpTrain,  W )+ b
        errTrain  = tf.reduce_mean((outTrain - yHatTrain)**2, name='errTrain')
        tf.summary.scalar('trainErr', errTrain)
        
    with tf.variable_scope('test'):
        yHatTest = tf.matmul(inpTest,  W )+ b
        errTest  = tf.reduce_mean((outTest - yHatTest)**2, name='errTest')
        tf.summary.histogram('testErrHist', (outTest - yHatTest)**2)
        tf.summary.scalar('testErr', errTest)

with tf.variable_scope('optimizer'):
    lr = tf.Variable(0.01, dtype=tf.float32, name='learningRate')
    opt = tf.train.AdamOptimizer(learning_rate=lr).minimize(errTrain)

init = tf.global_variables_initializer()

In [4]:
merged = tf.summary.merge_all()

In [5]:
folder = getNow()

with tf.Session() as sess:
    writer = tf.summary.FileWriter(folder, sess.graph)
    sess.run(init)
    
    for i in range(5001):
        sess.run( opt, feed_dict={
            inpTrain : X[tMask, :],
            outTrain : y[tMask, :],
            lr       : 0.01
             })
        
        if (i % 100) == 0:
            summary, errTrainVal, errTestVal = sess.run( [merged, errTrain, errTest], feed_dict={
                inpTrain : X[tMask, :],
                outTrain : y[tMask, :],
                inpTest : X[~tMask, :],
                outTest : y[~tMask, :]})
            
            writer.add_summary(summary, i)
            print('Training: {}, Testing: {}'.format(errTrainVal, errTestVal))
            
writer.close()

print('Final Values')
print('-'*30)
print('Training: {}, Testing: {}'.format(errTrainVal, errTestVal))
    
print('tensorboard --logdir={}'.format( folder ))

Training: 40.13312911987305, Testing: 40.21525192260742
Training: 19.561201095581055, Testing: 19.62839126586914
Training: 8.323736190795898, Testing: 8.374462127685547
Training: 3.0672836303710938, Testing: 3.103227138519287
Training: 1.005989909172058, Testing: 1.0307197570800781
Training: 0.33913254737854004, Testing: 0.3565252721309662
Training: 0.15956372022628784, Testing: 0.17266733944416046
Training: 0.11536970734596252, Testing: 0.12608012557029724
Training: 0.10107550024986267, Testing: 0.11036919057369232
Training: 0.09207272529602051, Testing: 0.10038135945796967
Training: 0.08391036093235016, Testing: 0.09140622615814209
Training: 0.0760197639465332, Testing: 0.08277406543493271
Training: 0.06840521097183228, Testing: 0.07445838302373886
Training: 0.06112736091017723, Testing: 0.06651479750871658
Training: 0.05423999950289726, Testing: 0.05899950489401817
Training: 0.0477842353284359, Testing: 0.05195698142051697
Training: 0.041789766401052475, Testing: 0.04541986808180809